# Kapitel 5

## Multi Layer Perceptron

In [8]:
# Für die mathematischen Operationen numpy importieren
import numpy as np
# Die Netzwerkklasse definieren
class MLP(object):
    # Die identische Funktion
    def func_id(self,x):
        return x
    # Eine weltberühmte Aktivierungsfunktion: Die Sigmoide
    def func_sigmoid(self,x):
        # Wichtig: Nicht math.exp sondern np.exp wegen array 
        # Operationen verwenden
        return 1.0 / (1.0 + np.exp(-x))
    def __init__(self, 
                 n_input_neurons=2,n_hidden_neurons=2,n_output_neurons=1,
                 weights=None,
                 *args, **kwargs):
        """ Initialisierung des Netzwerkes.
        "
        " Wir verwenden eine fixe I-H-O Struktur für den Anfang 
        " . (Input-Hidden-Output)
        " Die Anzahl der Neuronen ist flexibel
        " Weiters ist es möglich das Netzwerk mit Gewichten zu initialisieren
        "   [W_IH,W_HO]
        "
        """
        # Anzahl der Neuronen pro Layer
        self.n_input_neurons=n_input_neurons
        self.n_hidden_neurons=n_hidden_neurons
        self.n_output_neurons=n_output_neurons
        # Gewichtsinitialisierung
        self.weights = weights
        W_IH=[]
        W_HO=[]

        # Hier werden alle Daten zur Netzberechnung abgelegt
        self.network=[]
        # Input Layer + Bias Neuron: Spalten = o_i
        self.inputLayer = np.zeros((self.n_input_neurons+1,1))
        # Bias Neuron Output ist immer +1
        self.inputLayer[0] = 1.0 
        # Den Input Layer zum Netwerk hinzufügen
        self.network.append(self.inputLayer)
        # Weights von Input Layer zum Hidden Layer W_IH 
        # Neuron: Zeile x Spalten: Zeilen = # Hidden, Spalten = # Input
        # Nur Initialisieren, falls tatsächlich Gewichte vorhanden. 
        # Einfache Existenz Prüfung
        if weights:
            W_IH = self.weights[0]    
        else:
            W_IH = np.zeros((self.n_hidden_neurons+1,self.n_input_neurons+1))
        self.network.append(W_IH)
        # Hidden Layer + Bias Neuron: Spalten = net_i,a_i,o_i
        self.hiddenLayer = np.zeros((self.n_hidden_neurons+1,3))
        # Bias Neuron Output ist immer +1
        self.hiddenLayer[0] = 1.0 
        # Den Hidden Layer zum Netwerk hinzufügen
        self.network.append(self.hiddenLayer)
        # Weights von Hidden Layer zum Output Layer W_HO 
        # Neuron: Zeile x Spalten: Zeilen = # Output, Spalten = # Hidden
        if weights:
            W_HO = self.weights[1]
        else:
            W_HO = np.zeros((self.n_output_neurons+1,self.n_hidden_neurons+1))
        self.network.append(W_HO)
        # Output Layer + Bias Neuron: Spalten = net_i,a_i,o_i 
        self.outputLayer = np.zeros((self.n_output_neurons+1,3))
        # Bias Neuron Output = 0, da nicht relevant. 
        # Nur wegen einheitlicher Indizierung vorhanden
        self.outputLayer[0] = 0.0 
        # Den Output Layer zum Netwerk hinzufügen
        self.network.append(self.outputLayer)
    def print(self):
        print('Multi Layer Perceptron - Netzwerk Architektur')
        # Nur drei Nachkommastellen ausgeben
        np.set_printoptions(\
            formatter={'float': lambda x: "{0:0.3f}".format(x)})
        for idx,nn_part in enumerate(self.network):
            print(nn_part)
            print('----------v----------')  

    def predict(self,x):
        """ Für Eingabe x wird Ausgabe y_hat berechnet.
        "
        " Für den Vektor x wird eine Vorhersage berechnet und 
        " die Matrizenwerte der Layer (nicht Gewichte) werden angepasst
        "
        """
        # Input Layer
        # Die input Werte setzen: Alle Zeilen, Spalte 0
        self.network[0][:,0] = x
        # Hidden Layer
        # Start von Zeile 1 wegen Bias Neuron auf Index Position 0
        # net_i
        self.network[2][1:,0] = np.dot(self.network[1][1:,:],
                                       self.network[0][:,0])
        # a_i
        self.network[2][1:,1] = self.func_sigmoid(self.network[2][1:,0]) 
        # o_i
        self.network[2][1:,2] = self.func_id(self.network[2][1:,1]) 
        # Output Layer
        # Start von Zeile 1 wegen Bias Neuron auf 0
        # net_i
        self.network[4][1:,0] = np.dot(self.network[3][1:,:],
                                       self.network[2][:,2])
        # a_i
        self.network[4][1:,1] = self.func_sigmoid(self.network[4][1:,0]) 
        # o_i
        # self.network[4][1:,2] = self.func_id(self.network[4][1:,1]) 
        # o_i mit round
        self.network[4][1:,2] = np.round(self.func_id(self.network[4][1:,1]))         
        # Rückgabe Output Vektor
        return self.network[4][1:,2]
###########################
# Initialisierung der Gewichte
W_IH = np.matrix([[0.0,0.0,0.0],[-10,20.0,20.0],[30,-20.0,-20.0]])
W_HO = np.matrix([[0.0,0.0,0.0],[-30,20.0,20.0]])
weights=[]
weights.append(W_IH)
weights.append(W_HO)
nn = MLP(weights=weights)
# Netzwerk ausgeben
nn.print()
# Test
X=np.array([[1.0,1.0,1.0],[1.0,0,1.0],[1.0,1.0,0],[1.0,0,0]])
y=np.array([0,1.0,1.0,0])
print('Predict:')
for idx,x in enumerate(X):
    print('{} {} -> {}'.format(x,y[idx],nn.predict(x)))


Multi Layer Perceptron - Netzwerk Architektur
[[1.000]
 [0.000]
 [0.000]]
----------v----------
[[0.000 0.000 0.000]
 [-10.000 20.000 20.000]
 [30.000 -20.000 -20.000]]
----------v----------
[[1.000 1.000 1.000]
 [0.000 0.000 0.000]
 [0.000 0.000 0.000]]
----------v----------
[[0.000 0.000 0.000]
 [-30.000 20.000 20.000]]
----------v----------
[[0.000 0.000 0.000]
 [0.000 0.000 0.000]]
----------v----------
Predict:
[1.000 1.000 1.000] 0.0 -> [0.000]
[1.000 0.000 1.000] 1.0 -> [1.000]
[1.000 1.000 0.000] 1.0 -> [1.000]
[1.000 0.000 0.000] 0.0 -> [0.000]
